In [8]:
import os
import psycopg2
import csv

# Informações de conexão (substitua pelos seus dados)
db_host = "localhost"
db_name = "disaster"
db_user = "postgres"
db_password = ""

current_directory = os.getcwd()  
csv_folder = os.path.join(current_directory, "Trends")

# Mortality
#allowed_files = [
#    "2023-0054-TUR", "2023-0610-LBY", "2023-0054-SYR", "2023-0281-COD",
#    "2023-0600-MAR", "2023-0636-AFG", "2023-0428-IND", "2023-0095-MWI",
#    "2023-0691-NGA", "2023-0384-YEM",
#    "2022-0465-ITA", "2022-9436-UGA", "2022-0293-IND", "2022-0405-PAK", 
#    "2022-0342-AFG", "2022-0674-NGA", "2022-0205-ZAF", "2022-0204-PHL", 
#    "2022-0755-IDN", "2022-0089-BRA", "2021-0511-HTI", "2021-0435-IND", 
#    "2021-0369-CAN", "2021-0813-PHL", "2021-0428-CHN", "2021-0453-AFG", 
#    "2021-0096-USA", "2021-0072-IND", "2021-0390-USA", "2021-0173-IDN", 
#    "2020-0530-GBR", "2020-0530-FRA", "2020-0304-IND", "2020-0530-BEL", 
#    "2020-0297-NPL", "2020-0357-PAK", "2020-0530-NLD", "2020-0164-KEN", 
#    "2020-0322-CHN", "2020-0323-BGD"
#]

# Affected
#allowed_files = [
#    "2023-9494-IDN", "2023-0428-IND", "2023-0054-TUR", "2023-0054-SYR", 
#    "2023-0365-GTM", "2023-0792-IND", "2023-0807-TZA", "2023-0741-SOM", 
#    "2023-0095-MWI", "2023-0002-PHL", "2022-0405-PAK", "2022-9784-COD", 
#    "2022-9174-ETH", "2022-9787-NGA", "2022-9788-SDN", "2022-0340-BGD", 
#    "2022-9524-CHN", "2022-9254-NER", "2022-9781-BFA", "2022-0707-PHL", 
#    "2021-0428-CHN", "2021-9148-ZAF", "2021-9116-AFG", "2021-0813-PHL", 
#    "2021-9548-IRQ", "2020-9609-SOM", "2021-9546-ETH", "2021-9548-SYR", 
#    "2021-9510-IRN", "2020-0211-IND", "2020-0276-CHN", "2020-9569-MLI", 
#    "2020-0323-BGD", "2020-0501-PHL", "2020-0474-HND", "2020-0322-CHN", 
#    "2020-9570-NER", "2020-0463-PHL", "2020-9235-BFA"
#]

# Economic Losses
allowed_files = [
    "2023-0054-TUR", "2023-0464-CHN", "2023-9868-USA", "2023-0675-MEX", 
    "2023-0293-ITA", "2023-0054-SYR", "2023-0600-MAR", "2023-0610-LBY", 
    "2023-0122-USA", "2023-0524-USA", "2022-0614-USA", "2022-9209-USA", 
    "2022-0405-PAK", "2022-0153-JPN", "2022-9524-CHN", "2022-0103-AUS", 
    "2022-0316-CHN", "2022-0674-NGA", "2022-0293-IND", "2022-9211-BRA", 
    "2021-0551-USA", "2021-0411-DEU", "2021-0096-USA", "2021-0428-CHN", 
    "2021-9846-USA", "2021-0105-JPN", "2021-0825-FRA", "2021-0797-USA", 
    "2021-0832-USA", "2021-9078-CHN", "2020-0322-CHN", "2020-0211-IND", 
    "2020-0376-USA", "2020-0441-USA", "2020-0304-IND", "2020-0580-USA", 
    "2020-0438-USA", "2020-0296-JPN", "2020-0106-HRV", "2020-0474-HND"
]



try:
    # Conecta ao banco de dados
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    cur = conn.cursor()

    # Itera sobre os arquivos no diretório
    for filename in os.listdir(csv_folder):
        # Extrai o nome do arquivo sem a extensão
        file_title = os.path.splitext(filename)[0]

        # Verifica se o arquivo está na lista permitida
        if file_title in allowed_files and filename.endswith(".csv"):
            file_path = os.path.join(csv_folder, filename)
            
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                rows = list(reader)
                
                # Remove as 3 primeiras linhas
                rows = rows[3:]

                for row in rows:
                    # Adiciona o título do arquivo como o primeiro elemento (disno)
                    disno = file_title
                    week = row[0]  # Assume que a primeira coluna seja a data
                    popularity = row[1]  # Assume que a segunda coluna seja a popularidade
                    
                    # Limpa a coluna de popularidade, removendo < ou >
                    popularity = popularity.replace('<', '').replace('>', '')
                    
                    # Converte a popularidade para número, se necessário
                    try:
                        popularity = float(popularity)
                    except ValueError:
                        popularity = None  # Se falhar, considere None (valor nulo)

                    # factor = 1 #Top Mortality
                    # factor = 2 #Top Affected
                    factor = 3 #Top Economic
                    

                    # Insere os dados na tabela
                    cur.execute("""
                        INSERT INTO popularity (disno, week, popularity, factor)
                        VALUES (%s, %s, %s, %s)
                    """, (disno, week, popularity, factor))

    # Confirma as transações
    conn.commit()

except Exception as e:
    print(f"Erro: {e}")
    conn.rollback()

finally:
    # Fecha a conexão com o banco de dados
    cur.close()
    conn.close()


In [21]:
import psycopg2
import pandas as pd

# Informações de conexão ao banco de dados
db_host = "localhost"
db_name = "disaster"
db_user = "postgres"
db_password = ""

# Conectar ao banco de dados
conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
cursor = conn.cursor()

# 1. Obter disnos únicos da tabela popularity
unique_disno_query = """
    SELECT DISTINCT disno 
    FROM popularity;
"""
unique_disno_df = pd.read_sql(unique_disno_query, conn)

# 2. Preparar lista de disnos únicos
unique_disnos = unique_disno_df['disno'].tolist()

# Dicionário para armazenar resultados
results = []

# 3. Processar cada disno único
for disno in unique_disnos:
    # Obter a data de início do desastre correspondente
    disaster_query = f"""
        SELECT start_year, start_month, start_day 
        FROM disasters 
        WHERE DisNo = '{disno}';
    """
    disaster_df = pd.read_sql(disaster_query, conn)
    
    # Verifica se o desastre existe
    if not disaster_df.empty:
        # Certifique-se de que os valores são inteiros
        start_year = int(disaster_df['start_year'].iloc[0])
        
        # Verificar se start_month está vazio e definir como 1 se necessário
        start_month_value = disaster_df['start_month'].iloc[0]
        start_month = int(start_month_value) if start_month_value is not None else 1
        
        # Verificar se start_day está vazio e definir como 1 se necessário
        start_day_value = disaster_df['start_day'].iloc[0]
        start_day = int(start_day_value) if start_day_value is not None else 1

        # Calcular o intervalo de 13 meses
        try:
            # Use o construtor Timestamp corretamente
            start_date = pd.Timestamp(year=start_year, month=start_month, day=start_day) - pd.DateOffset(months=1)
            end_date = start_date + pd.DateOffset(months=13)

            # 4. Consultar a tabela popularity para o disno dentro do intervalo
            popularity_query = f"""
                SELECT SUM(popularity) AS total_popularity,
                       COUNT(CASE WHEN popularity > 0 THEN 1 END) AS non_zero_weeks_count
                FROM popularity
                WHERE disno = '{disno}'
                AND week BETWEEN '{start_date.strftime('%Y-%m-%d')}' AND '{end_date.strftime('%Y-%m-%d')}';
            """
            popularity_df = pd.read_sql(popularity_query, conn)

            # Adicionar resultados ao dicionário
            results.append({
                'disno': disno,
                'total_popularity': popularity_df['total_popularity'].iloc[0] or 0,
                'non_zero_weeks_count': popularity_df['non_zero_weeks_count'].iloc[0] or 0,
            })
        except Exception as e:
            print(f"Error processing disno {disno}: {e}")

# 5. Criar um DataFrame para armazenar os resultados
results_df = pd.DataFrame(results)

# 6. Inserir os resultados na nova tabela
insert_query = """
    INSERT INTO disaster_popularity_summary (disno, total_popularity, non_zero_weeks_count) 
    VALUES (%s, %s, %s)
    ON CONFLICT (disno) DO UPDATE 
    SET total_popularity = EXCLUDED.total_popularity,
        non_zero_weeks_count = EXCLUDED.non_zero_weeks_count;
"""

# Executar inserção
for _, row in results_df.iterrows():
    try:
        cursor.execute(insert_query, (row['disno'], row['total_popularity'], row['non_zero_weeks_count']))
    except Exception as e:
        print(f"Error inserting disno {row['disno']}: {e}")
        conn.rollback()  # Reverter a transação em caso de erro

# Commit das alterações
conn.commit()

# Fechar a conexão com o banco de dados
cursor.close()
conn.close()


/var/folders/0l/t94s0yfj5rj_nsth6nh161r00000gn/T/ipykernel_11320/2307002972.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  unique_disno_df = pd.read_sql(unique_disno_query, conn)
/var/folders/0l/t94s0yfj5rj_nsth6nh161r00000gn/T/ipykernel_11320/2307002972.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  disaster_df = pd.read_sql(disaster_query, conn)
/var/folders/0l/t94s0yfj5rj_nsth6nh161r00000gn/T/ipykernel_11320/2307002972.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popularity_df = pd.read_sql(popularity